In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import plotly.express as px
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from scipy.stats import mannwhitneyu

In [2]:
raw_data = pd.read_csv("processed_data/out19_all_companies_rank_fin.csv")


In [3]:
raw_data.columns

Index(['Id', 'Ticker', 'Company Name', 'Country', 'GICS Sector',
       'fullTimeEmployees', 'Original Currency', 'ESG Rank Year', 'ESG Rank',
       'Average Rank', 'Years Appeared', 'Fin Metrics Year',
       'Stock_Performance per', 'ReportDate', 'TotalRevenueIncr per',
       'NetIncomeIncr per', 'OperatingIncomeIncr per', 'GrossProfitIncr per',
       'DilutedEPSIncr per', 'TotalRevenue', 'NetIncome', 'OperatingIncome',
       'GrossProfit', 'DilutedEPS', 'Net Margin per', 'Operating Margin per',
       'Gross Margin per', 'ToralRevenue per employee',
       'NetIncome per employee'],
      dtype='object')

In [4]:

df = pd.read_csv("processed_data/out19_all_companies_rank_fin.csv")

df["ESG_Ranked"] = df["ESG Rank"].notna()*1.0

results = []

for metric in ["Stock_Performance per", "fullTimeEmployees", 'TotalRevenueIncr per',
       'NetIncomeIncr per', 'OperatingIncomeIncr per', 'GrossProfitIncr per',
       'DilutedEPSIncr per', 'TotalRevenue', 'NetIncome', 'OperatingIncome',
       'GrossProfit', 'DilutedEPS', 'Net Margin per', 'Operating Margin per',
       'Gross Margin per']:

    # convert to numeric, stripping % if present
    df[metric] = df[metric].replace("%","", regex=True)
    df[metric] = pd.to_numeric(df[metric], errors="coerce")


    
    for sector in df["GICS Sector"].dropna().unique():
        sector_df = df[df["GICS Sector"] == sector]
        
        ranked = sector_df[sector_df["ESG_Ranked"] == 1]
        unranked = sector_df[sector_df["ESG_Ranked"] == 0]
        

        
        ranked_metric = ranked[metric].dropna()
        unranked_metric = unranked[metric].dropna()

        
        avg_ranked = ranked_metric.mean()
        avg_unranked = unranked_metric.mean()

        median_ranked = ranked_metric.median()
        median_unranked = unranked_metric.median()

        count_ranked = len(ranked_metric)
        count_unranked = len(unranked_metric)
        
        if count_ranked > 1 and count_unranked > 1:
            try:
                t_stat, p_value = stats.ttest_ind(ranked_metric, unranked_metric, equal_var=False)
            except Exception:
                t_stat, p_value = None, None
            try:
                mw_stat, mw_p_value = mannwhitneyu(ranked_metric, unranked_metric, alternative="two-sided")
            except Exception:
                mw_stat, mw_p_value = None, None
        else:
            t_stat, p_value = None, None
            mw_stat, mw_p_value = None, None

        results.append({
            "Sector": sector,
            "Metric": metric,
            "Ranked Avg metric": avg_ranked,
            "Unranked Avg metric": avg_unranked,
            "Ranked Median metric": median_ranked,
            "Unranked Median metric": median_unranked,
            "Ranked Count": count_ranked,
            "Unranked Count": count_unranked,
            "T-Statistic": t_stat,
            "T-Test P-Value": p_value,
            "MWU P-Value": mw_p_value
        })

results_df = pd.DataFrame(results)
results_df["T-Test Significant"] = results_df["T-Test P-Value"] < 0.05
results_df["MWU Significant"] = results_df["MWU P-Value"] < 0.05


# show only rows where *either* test is significant
results_filt_df = results_df[
    (results_df["T-Test Significant"]) | (results_df["MWU Significant"])
    ].sort_values(by=["Sector", "T-Test P-Value"])


results_filt_df.to_csv("outputs/s11_esg_sector_comparison.csv", index=False)
results_df.to_csv("outputs/s11_esg_sector_comparison_ALL_data.csv", index=False)

In [5]:
import pandas as pd
from scipy import stats
from scipy.stats import mannwhitneyu
from sklearn.linear_model import LinearRegression


df = pd.read_csv("processed_data/out19_all_companies_rank_fin.csv")


df["ESG_Ranked"] = df["ESG Rank"].notna() * 1.0


results = []


metrics = [
    "Stock_Performance per", "fullTimeEmployees", 'TotalRevenueIncr per',
    'NetIncomeIncr per', 'OperatingIncomeIncr per', 'GrossProfitIncr per',
    'DilutedEPSIncr per', 'TotalRevenue', 'NetIncome', 'OperatingIncome',
    'GrossProfit', 'DilutedEPS', 'Net Margin per', 'Operating Margin per',
    'Gross Margin per'
]

# metrics to normalize using revenue
normalize_metrics = ['NetIncome', 'OperatingIncome', 'GrossProfit']

# clean/ convert all metric columns
for metric in metrics:
    df[metric] = df[metric].replace("%", "", regex=True)
    df[metric] = pd.to_numeric(df[metric], errors="coerce")


for metric in metrics:
    for sector in df["GICS Sector"].dropna().unique():
        sector_df = df[df["GICS Sector"] == sector].copy()

        # if metric needs normalization, regress on TotalRevenue and use residuals
        if metric in normalize_metrics:
            valid_mask = sector_df[["TotalRevenue", metric]].notna().all(axis=1)
            
            X = sector_df.loc[valid_mask, ["TotalRevenue"]]
            Y = sector_df.loc[valid_mask, [metric]]
            
            if len(Y) > 1:
                lr = LinearRegression()
                lr.fit(X, Y)
                predictions = lr.predict(X)
                sector_df.loc[valid_mask, metric] = Y.values.flatten() - predictions.flatten()


        # split into ESG-ranked and unranked
        ranked = sector_df[sector_df["ESG_Ranked"] == 1]
        unranked = sector_df[sector_df["ESG_Ranked"] == 0]

        
        ranked_metric = ranked[metric].dropna()
        unranked_metric = unranked[metric].dropna()

       
        avg_ranked = ranked_metric.mean()
        avg_unranked = unranked_metric.mean()
        median_ranked = ranked_metric.median()
        median_unranked = unranked_metric.median()
        count_ranked = len(ranked_metric)
        count_unranked = len(unranked_metric)

        #  statistical tests
        if count_ranked > 1 and count_unranked > 1:
            try:
                t_stat, p_value = stats.ttest_ind(ranked_metric, unranked_metric, equal_var=False)
            except Exception:
                t_stat, p_value = None, None
            try:
                mw_stat, mw_p_value = mannwhitneyu(ranked_metric, unranked_metric, alternative="two-sided")
            except Exception:
                mw_stat, mw_p_value = None, None
        else:
            t_stat, p_value = None, None
            mw_stat, mw_p_value = None, None

        results.append({
            "Sector": sector,
            "Metric": metric,
            "Ranked Avg metric": avg_ranked,
            "Unranked Avg metric": avg_unranked,
            "Ranked Median metric": median_ranked,
            "Unranked Median metric": median_unranked,
            "Ranked Count": count_ranked,
            "Unranked Count": count_unranked,
            "T-Statistic": t_stat,
            "T-Test P-Value": p_value,
            "MWU P-Value": mw_p_value,
            "Normalized By Revenue": metric in normalize_metrics
        })

results_df = pd.DataFrame(results)
results_df["T-Test Significant"] = results_df["T-Test P-Value"] < 0.05
results_df["MWU Significant"] = results_df["MWU P-Value"] < 0.05


# show only rows where either test is significant
results_filt_df = results_df[
    (results_df["T-Test Significant"]) | (results_df["MWU Significant"])
].sort_values(by=["Sector", "T-Test P-Value"])


results_filt_df.to_csv("outputs/s11_sector_comparison_normalized.csv", index=False)
results_df.to_csv("outputs/s11_esg_sector_comparison_normalized_ALL_data.csv", index=False)


,Sector,Metric,Ranked Avg metric,Unranked Avg metric,Ranked Median metric,Unranked Median metric,Ranked Count,Unranked Count,T-Statistic,P-Value
156,Communication Services,Gross Margin per,6.591143e+01,5.254114e+01,6.813500e+01,5.541000e+01,64,446,3.765228,0.000218
99,Consumer Discretionary,OperatingIncome,1.402267e+09,5.808587e+09,7.257044e+08,2.229314e+09,110,590,-3.360476,0.000887
11,Consumer Discretionary,fullTimeEmployees,8.867450e+04,1.852104e+05,4.462700e+04,6.719500e+04,110,590,-3.269454,0.001135
77,Consumer Discretionary,TotalRevenue,1.757152e+10,6.444570e+10,8.821003e+09,2.361255e+10,110,590,-3.248498,0.001303
88,Consumer Discretionary,NetIncome,1.125449e+09,4.688705e+09,4.823729e+08,1.456063e+09,110,590,-2.980324,0.003134
110,Consumer Discretionary,GrossProfit,5.638657e+09,1.898841e+10,3.360039e+09,9.494000e+09,110,590,-2.602242,0.009758
148,Consumer Staples,Operating Margin per,1.001893e+01,1.727650e+01,1.052500e+01,1.603500e+01,84,506,-2.934773,0.003673
104,Consumer Staples,OperatingIncome,2.544103e+09,5.583068e+09,9.224871e+08,3.224600e+09,84,506,-2.627470,0.009176
126,Consumer Staples,DilutedEPS,1.999536e+00,3.749495e+00,2.245000e+00,2.765000e+00,84,506,-2.559126,0.011130
93,Consumer Staples,NetIncome,1.704416e+09,3.742015e+09,6.813000e+08,2.155000e+09,84,506,-2.434175,0.015682


In [6]:
df = pd.read_csv("processed_data/out19_all_companies_rank_fin.csv")
model = smf.ols(formula = "Q('ESG Rank')~Q('DilutedEPSIncr per')", data = df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Q('ESG Rank')   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     2.582
Date:                Tue, 19 Aug 2025   Prob (F-statistic):              0.109
Time:                        12:10:24   Log-Likelihood:                -1338.4
No. Observations:                 281   AIC:                             2681.
Df Residuals:                     279   BIC:                             2688.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                  51.3312      1.698     30.224      0.000      47.988      54.674
Q('DilutedEPSIncr per')     0.0065      0.004      1.607      0.109      -0.001       0.014
==============================================================================
Omnibus:                      123.841   Durbin-Watson:                   1.344
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               16.152
Skew:                          -0.052   Prob(JB):                     0.000311
Kurtosis:                       1.830   Cond. No.                         422.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [7]:
model = smf.ols(formula = "Q('Net Margin per')~Q('Stock_Performance per')", data = df[df["GICS Sector"] == "Information Technology"]).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                            
===============================================================================
Dep. Variable:     Q('Net Margin per')   R-squared:                       0.085
Model:                             OLS   Adj. R-squared:                  0.082
Method:                  Least Squares   F-statistic:                     26.62
Date:                 Tue, 19 Aug 2025   Prob (F-statistic):           4.64e-07
Time:                         12:10:26   Log-Likelihood:                -1315.0
No. Observations:                  289   AIC:                             2634.
Df Residuals:                      287   BIC:                             2641.
Df Model:                            1                                         
Covariance Type:             nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                     10.4669      1.420      7.372      0.000       7.672      13.261
Q('Stock_Performance per')     0.1350      0.026      5.159      0.000       0.083       0.187
==============================================================================
Omnibus:                      310.168   Durbin-Watson:                   1.426
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            14013.127
Skew:                          -4.485   Prob(JB):                         0.00
Kurtosis:                      35.913   Cond. No.                         57.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""